In [1]:
import itertools
import json
import pickle
import re

import numpy as np
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
with open("../../data/pseudowords/CoMaPP_all.json") as json_file:
    data = json.load(json_file)
    
data = [{"example": d["target1"], "cue": " ".join(d["target1"].split()[:d["query_idx"]]), "pseudoword": d["label"]} for d in data]
df = pd.DataFrame.from_dict(data).drop_duplicates(ignore_index=True)
df

,example,cue,pseudoword
0,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
1,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
2,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
3,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
4,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
12128,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
12129,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
12130,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99
12131,"An seiner Hand geht ein Mädchen , einen Kopf k...","An seiner Hand geht ein Mädchen , einen Kopf k...",er99


In [3]:
df['index'] = df['pseudoword'].str.extract('(\d+)').astype(int)
df.set_index('index', inplace=True)

df

,example,cue,pseudoword
index,,,
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99


In [4]:
contextleft = pd.read_pickle("../../data/pseudowords/contextleft_text.pickle")

def update_cue(row):
    if row['cue'] == '':  # if the string in cue is empty
        # match the index of row with contextleft['construction_id'] and match contextleft['text'] with row['example'] and create matching_entry
        matching_entry = contextleft.loc[(contextleft['construction_id'] == row.name) & (contextleft['text'] == row['example']), 'contextleft'].tolist()
        if len(matching_entry) > 0:
            return [matching_entry[0] + " " + row['example'], matching_entry[0]]
    return row[['example', 'cue']]

# Add the left context if there is no cue up until the pseudoword.
df[["example", "cue"]] = df.apply(update_cue, axis=1)
df

,example,cue,pseudoword
index,,,
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99


In [5]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'construction'}, inplace=True)

result_df = df.groupby(['construction', 'pseudoword']).agg({'example': list, 'cue': list})

result_df

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  
construction pseudoword                                                     
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...  
             erst5       [Trainer Lucien Favre hatte schon seine beiden...  
             gar5        [Es hat Afghanistan nicht stabilisiert, Es hat...  
             nicht5      [Es, Es hat Afghanistan, Dass, Dass es in der ...  
             recht5      [Trainer Lucien Favre hatte schon seine beiden...  
...                                                                    ...  
1884         Gold1884    [Schweigen ist Silber , reden ist, Schweigen i...  
             Silber1884  [Schweigen ist, Schweigen ist, Angeben ist, Sc...  
             ist1884     [Schweigen, Schweigen, Angeben, Schweigen, " Q...  
1986         kaum1986    [Die Vorhut vor 20.000 Jahren, Die Vorhut vor ...  
1987         sehr1987    [Solche Menschen können, Solche Menschen könne...  

[562 rows x 2 columns]

In [6]:
with open("../../out/definitions.pickle", "rb") as definitions_file:
    definitions = pd.DataFrame.from_dict(pickle.load(definitions_file), orient="index", columns=["definition"])
    
definitions

,definition
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio..."
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de..."
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru..."
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""..."
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ..."
...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g..."
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e..."
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han..."
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z..."


In [7]:
examples = pd.merge(result_df, definitions, how="inner", left_on="construction", right_index=True)  # TODO: Aussortierte Definitions betrachten!
examples

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert, Es hat...   
             nicht5      [Es, Es hat Afghanistan, Dass, Dass es in der ...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist, Schweigen i...   
             Silber1884  [Schweigen ist, Schweigen ist, Angeben ist, Sc...   
             ist1884     [Schweigen, Schweigen, Angeben, Schweigen, " Q...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren, Die Vorhut vor ...   
1987         sehr1987    [Solche Menschen können, Solche Menschen könne...   

                                                                definition  
construction pseudoword                                                     
5            Und5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             erst5       Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             gar5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             nicht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             recht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
...                                                                    ...  
1884         Gold1884    Die "Intensivierung_Komparativ:Xist_SilberYist...  
             Silber1884  Die "Intensivierung_Komparativ:Xist_SilberYist...  
             ist1884     Die "Intensivierung_Komparativ:Xist_SilberYist...  
1986         kaum1986    Die Konstruktion "Relativierung:kaumADJ" gehör...  
1987         sehr1987    Die Konstruktion "Intensivierung:sehrV" gehört...  

[562 rows x 3 columns]

##### Generieren neuer Sätze:

In [8]:
pseudowords = np.load("../../out/pseudowords_comapp.npy")
pseudowords

array([[-3.66079696e-02, -1.11742102e-05,  5.81527799e-02, ...,
         6.40575811e-02,  2.65836474e-02, -1.56662956e-01],
       [-5.31983189e-02, -1.80779640e-02, -6.33598045e-02, ...,
         2.10250225e-02, -6.49476610e-03,  5.84359504e-02],
       [-9.96073708e-02,  3.98329496e-02, -1.11172702e-02, ...,
         1.56649694e-01,  2.40581349e-01, -9.96441580e-03],
       ...,
       [-1.07323132e-01,  9.40747932e-02,  8.12232345e-02, ...,
        -1.49555236e-01,  8.22361112e-02, -5.65737225e-02],
       [-4.59361728e-03,  1.85684375e-02,  2.77958419e-02, ...,
         2.84781586e-02,  2.85724066e-02, -5.74652366e-02],
       [-1.82020497e-02, -2.03991937e-03, -3.41283977e-02, ...,
        -2.80237161e-02, -2.23183036e-02,  5.36667835e-03]])

In [9]:
with open("../../data/pseudowords/CoMaPP_all.json") as json_file:
    json_data = json.load(json_file)
json_data

[{'label': 'geschweige10',
  'target1': 'Und dann ist da noch das generelle Problem mit Hamas , dass nicht jeder Sprecher und Führer , der redet , auch etwas zu sagen , geschweige denn das letzte Wort hat .',
  'target1_idx': 26,
  'query': 'Und dann ist da noch das generelle Problem mit Hamas , <mask> nicht jeder Sprecher und Führer , der redet , auch etwas zu sagen , geschweige denn das letzte Wort hat .',
  'query_idx': 26},
 {'label': 'denn10',
  'target1': 'Und dann ist da noch das generelle Problem mit Hamas , dass nicht jeder Sprecher und Führer , der redet , auch etwas zu sagen , geschweige denn das letzte Wort hat .',
  'target1_idx': 27,
  'query': 'Und dann ist da noch das generelle Problem mit Hamas , <mask> nicht jeder Sprecher und Führer , der redet , auch etwas zu sagen , geschweige denn das letzte Wort hat .',
  'query_idx': 27},
 {'label': 'geschweige10',
  'target1': 'Und dann ist da noch das generelle Problem mit Hamas , dass nicht jeder Sprecher und Führer , der red

In [10]:
json_data.sort(key=lambda x: x["label"])  # Grouping doesn't work without sorting first!
json_data = [list(group) for _, group in itertools.groupby(json_data, key=lambda x: x["label"])]
mbart_tokens = [d[0]['label'] for d in json_data]  # TODO Check whether the labels are in correct order!

mbart_tokens, len(mbart_tokens)

(['""Was13',
  '"647',
  '"Wir-äh-spielen-äh-in-der-äh-Champions-League647',
  '(1597',
  '(1600',
  '(1602',
  '(1637',
  '(1641',
  '(1643',
  '(379',
  '(579',
  '(581',
  '(584',
  '(590',
  '(592',
  '(600',
  '(886',
  '(892',
  '(900',
  '(905',
  '(907',
  '(909',
  '(917',
  '(919',
  '(921',
  ')1597',
  ')1600',
  ')1602',
  ')1637',
  ')1641',
  ')1643',
  ')1792',
  ')379',
  ')579',
  ')581',
  ')584',
  ')590',
  ')592',
  ')600',
  ')886',
  ')892',
  ')900',
  ')905',
  ')907',
  ')909',
  ')917',
  ')919',
  ')921',
  ')«579',
  ',1459',
  ',973',
  '-128',
  '-651',
  '-654',
  '-875',
  '-973',
  ':595',
  ':875',
  ':973',
  'Abstand683',
  'Allein20',
  'Aller1630',
  'Als1315',
  'Als133',
  'Als1770',
  'Am488',
  'Am492',
  'Am500',
  'Amerika605',
  'Anstatt320',
  'Art129',
  'Arzt1509',
  'Augenblick1301',
  'Ausmaß1777',
  'BRUTAL1503',
  'Besser1762',
  'Bis559',
  'Brutal1503',
  'Buche1346',
  'Das1313',
  'Das1461',
  'Dass21',
  'Dasselbe104',
  'Der13

Load the vanilla mBART-50 model:

In [11]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50", return_dict=True)
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50", src_lang="de_DE", tgt_lang="de_DE")
model.model.encoder.embed_tokens

Embedding(250054, 1024, padding_idx=1)

Add to existing embeddings:

In [12]:
combined_embeddings = torch.cat((model.model.encoder.embed_tokens.weight, torch.tensor(pseudowords)), dim=0)
model.model.encoder.embed_tokens = torch.nn.Embedding.from_pretrained(combined_embeddings)
model.model.encoder.embed_tokens

Embedding(250327, 1024)

Add to existing tokens:

In [13]:
tokenizer.add_tokens(mbart_tokens[:len(pseudowords)])
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 250327. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(250327, 1024)

Complete the cues:

In [15]:
# TODO: input: cue + pseudoword -> output: generate with len(example)

def complete_cues(row):    
    input_text = row["cue"] + " " + row["pseudoword"]
    tokenized_text = tokenizer.tokenize(input_text)
    
    # convert the tokens to input_ids:
    input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    input_ids = torch.tensor([input_ids])
    
    target_length = 2 * len(row["example"])  # allow double the length of the original sentence
    
    outputs = tokenizer(input_ids, return_tensors="pt")
    outputs = model.generate(outputs["input_ids"], max_length=target_length, num_return_sequences=1,
                             num_beams=20, output_scores=True, return_dict_in_generate=True)
    output_text = tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)
    score = torch.exp(outputs.sequences_scores)
    
    return [output_text, score]
    
examples[["generated", "score"]] = examples[["example", "cue", "pseudoword"]].progress_apply(complete_cues)  # TODO use "pseudoword" as index

KeyError: "['pseudoword'] not in index"

#### Anzahl neuer Sätze

In [ ]:
examples.to_csv(f"../../out/comapp/data.tsv", sep="\t", decimal=",")
examples.to_excel(f"../../out/comapp/data.xlsx")